In [22]:
import tengp
from experiment_settings import korns12_funset
import symreg

In [30]:
exact_solution = [0, 0, 0, 2,
                  0, 0, 0, 2.1,
                  0, 0, 0, 9.8,
                  3, 7, 3, 0,
                  6, 8, 0, 0,
                  3, 6, 9, 0,
                  0, 0, 0, 1.3,
                  3, 2, 11, 0,
                  5, 12, 0, 0,
                  3, 10, 13, 0,
                  2, 5, 14, 0,
                  15]

In [31]:
paramsKorns = tengp.Parameters(5, 1, 1, 11, korns12_funset, real_valued=True)

bounds = tengp.individual.IndividualBuilder(paramsKorns).create().bounds

individual = cparamsKorns.individual_class(exact_solution, bounds, paramsKorns)

In [32]:
import random

x_train, y_train, x_test, y_test = symreg.get_benchmark_korns(random, 12)

y_pred = individual.transform(x_train)

In [33]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_pred, y_train)

1276859.3844063533

# Try to optimize with PSO

In [10]:
import pygmo as pg

class cost_function:
    def __init__(self, X, Y, params, bounds):
        self.params = params
        self.bounds = bounds
        self.X = X
        self.Y = Y
    
    def fitness(self, x):      
        individual = tengp.individual.NPIndividual(list(x), self.bounds, self.params)
        
        pred = individual.transform(self.X)
        
        try:
            return [mean_squared_error(pred, self.Y)]
        except ValueError:
            return [10000000000]
        
    def get_bounds(self):
        self.bounds[0][-1] = 12
        return self.bounds

In [11]:
prob = pg.problem(cost_function(x_train, y_train, cparamsKorns, bounds))



algo = pg.algorithm(pg.pso(gen=250))

algo.set_verbosity(2)

pop = pg.population(prob, 20)

pop = algo.evolve(pop)

print(pop.champion_f[0])

/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:35: RuntimeWarning: invalid value encountered in sqrt
  return  np.sqrt(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:38: RuntimeWarning: overflow encountered in exp
  return np.exp(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:11: RuntimeWarning: invalid value encountered in greater
  return np.log(x, out=np.copy(x), where=x>0)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:17: RuntimeWarning: invalid value encountered in cos
  return np.cos(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:29: RuntimeWarning: invalid value encountered in tan
  return np.tan(x)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:103: RuntimeWarning: overflow encountered in multiply
  u = a*f_u(L, U)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:104: RuntimeWar

KeyboardInterrupt: 

# Random walk

In [12]:
import random

random.seed(42)

STEP_SIZE = 0.1

NUMBER_OF_STEPS = 1000

start_point = exact_solution
random_walk = [start_point]
new_point = start_point[:]

i = 0
while i < NUMBER_OF_STEPS:
    #print(i, end=',')
    index = random.randint(0, len(start_point) - 1)
    
    move = random.choice([-STEP_SIZE, STEP_SIZE])  
    
    if (new_point[index] + move) < 0 or (new_point[index] + move) > bounds[1][index]:        
        continue
    
    
    new_point[index] += move
    
    
    if new_point not in random_walk:    
        random_walk.append(new_point)
        new_point = new_point[:]
        i += 1
    
    

IndexError: list index out of range

In [20]:
len(start_point)

45

In [ ]:
fitnesses = []

for genes in random_walk:
    individual = cparamsKorns.individual_class(genes, bounds, cparamsKorns)
    output = individual.transform(x_train)
    fitnesses.append(mean_squared_error(output, y_train))

In [ ]:
fitnesses

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

distances = euclidean_distances(random_walk, [start_point])

distances = distances.flatten()

In [ ]:
import matplotlib.pyplot as plt
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

output_notebook()

In [ ]:
p = figure(plot_width=800, plot_height=400, title='random walk fitnesses')
x_axis = list(range(len(fitnesses)))
p.line(, fitnesses)

show(p)

In [ ]:
fitnesses

In [ ]:
p = figure(plot_width=800, plot_height=400, title='random walk distance from start point')
p.line(list(range(len(fitnesses))), distances)
show(p)